In [19]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importação bibliotecas

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

Upload base de dados

In [21]:
dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/acoes.csv')
dataset.head()

,Date,ABEV,ODPV,CCR,PETBR,BB,BOVA
0,2015-01-02,16.020000,9.37,14.72,9.36,22.650000,47.259998
1,2015-01-05,15.730000,9.62,14.30,8.56,22.180000,46.320000
2,2015-01-06,16.330000,9.55,14.51,8.28,22.490000,46.580002
3,2015-01-07,16.559999,9.69,15.60,8.67,23.480000,48.150002
4,2015-01-08,16.540001,9.66,15.78,9.23,23.559999,48.509998


Alocação aleatória de ativos

In [22]:
def alocacao_ativos(dataset, dinheiro_total, seed = 0, melhores_pesos = []):
  dataset = dataset.copy()

  if seed != 0:
    np.random.seed(seed)

  if len(melhores_pesos) > 0:
    pesos = melhores_pesos
  else:  
    pesos = np.random.random(len(dataset.columns) - 1)
    #print(pesos, pesos.sum())
    pesos = pesos / pesos.sum()
    #print(pesos, pesos.sum())

  colunas = dataset.columns[1:]
  #print(colunas)
  for i in colunas:
    dataset[i] = (dataset[i] / dataset[i][0])

  for i, acao in enumerate(dataset.columns[1:]):
    #print(i, acao)
    dataset[acao] = dataset[acao] * pesos[i] * dinheiro_total
  
  dataset['soma valor'] = dataset.sum(axis = 1)

  datas = dataset['Date']
  #print(datas)

  dataset.drop(labels = ['Date'], axis = 1, inplace = True)
  dataset['taxa retorno'] = 0.0

  for i in range(1, len(dataset)):
    dataset['taxa retorno'][i] = ((dataset['soma valor'][i] / dataset['soma valor'][i - 1]) - 1) * 100

  acoes_pesos = pd.DataFrame(data = {'Ações': colunas, 'Pesos': pesos * 100})

  return dataset, datas, acoes_pesos, dataset.loc[len(dataset) - 1]['soma valor']

In [23]:
dataset, datas, acoes_pesos, soma_valor = alocacao_ativos(pd.read_csv('acoes.csv'), 5000, 10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.


Visualização portfólio

In [25]:
fig = px.line(x = datas, y = dataset['taxa retorno'], title = 'Retorno diário do portfólio')
fig.show()

In [26]:
fig = px.line(title = 'Evolução do patrimônio')
for i in dataset.drop(columns = ['soma valor', 'taxa retorno']).columns:
  fig.add_scatter(x = datas, y = dataset[i], name = i)
fig.show()

In [27]:
fig = px.line(x = datas, y = dataset['soma valor'], title = 'Evolução do patrimônio')
fig.show()

Retorno Acumulado no período

In [28]:
dataset.loc[len(dataset) - 1]['soma valor'] / dataset.loc[0]['soma valor'] - 1

0.8044784257243494

Desvio-Padrão do Retorno

In [29]:
dataset['taxa retorno'].std()

2.0486483870008807

Sharpe-Ratio

In [30]:
(dataset['taxa retorno'].mean() / dataset['taxa retorno'].std()) * np.sqrt(246)

0.398093163751987

In [32]:
dinheiro_total = 5000
soma_valor - dinheiro_total

4022.3921286217465

Taxa selic anual no período

In [37]:
taxa_selic_2015 = 12.75
taxa_selic_2016 = 14.25
taxa_selic_2017 = 12.25
taxa_selic_2018 = 6.50
taxa_selic_2019 = 5.0
taxa_selic_2020 = 2.0

taxa_selic_historico = np.array([12.75, 14.25, 12.25, 6.5, 5.0, 2.0])
taxa_selic_historico.mean() / 100

0.08791666666666666

In [38]:
import sys
1 - sys.maxsize

def alocacao_portfolio(dataset, dinheiro_total, sem_risco, repeticoes):
  dataset = dataset.copy()
  dataset_original = dataset.copy()

  lista_retorno_esperado = []
  lista_volatilidade_esperada = []
  lista_sharpe_ratio = []

  melhor_sharpe_ratio = 1 - sys.maxsize
  melhores_pesos = np.empty
  melhor_volatilidade = 0
  melhor_retorno = 0
  
  for _ in range(repeticoes):
    pesos = np.random.random(len(dataset.columns) - 1)
    pesos = pesos / pesos.sum()

    for i in dataset.columns[1:]:
      dataset[i] = dataset[i] / dataset[i][0]

    for i, acao in enumerate(dataset.columns[1:]):
      dataset[acao] = dataset[acao] * pesos[i] * dinheiro_total

    dataset.drop(labels = ['Date'], axis = 1, inplace=True)

    retorno_carteira = np.log(dataset / dataset.shift(1))
    matriz_covariancia = retorno_carteira.cov()

    dataset['soma valor'] = dataset.sum(axis = 1)
    dataset['taxa retorno'] = 0.0

    for i in range(1, len(dataset)):
      dataset['taxa retorno'][i] = np.log(dataset['soma valor'][i] / dataset['soma valor'][i - 1])

    #sharpe_ratio = (dataset['taxa retorno'].mean() - sem_risco) / dataset['taxa retorno'].std() * np.sqrt(246)
    retorno_esperado = np.sum(dataset['taxa retorno'].mean() * pesos) * 246
    volatilidade_esperada = np.sqrt(np.dot(pesos, np.dot(matriz_covariancia * 246, pesos)))
    sharpe_ratio = (retorno_esperado - sem_risco) / volatilidade_esperada

    if sharpe_ratio > melhor_sharpe_ratio:
      melhor_sharpe_ratio = sharpe_ratio
      melhores_pesos = pesos
      melhor_volatilidade = volatilidade_esperada
      melhor_retorno = retorno_esperado

    lista_retorno_esperado.append(retorno_esperado)
    lista_volatilidade_esperada.append(volatilidade_esperada)
    lista_sharpe_ratio.append(sharpe_ratio)
    
    dataset = dataset_original.copy()

  return melhor_sharpe_ratio, melhores_pesos, lista_retorno_esperado, lista_volatilidade_esperada, lista_sharpe_ratio, melhor_volatilidade, melhor_retorno

In [39]:
sharpe_ratio, melhores_pesos, ls_retorno, ls_volatilidade, ls_sharpe_ratio, melhor_volatilidade, melhor_retorno = alocacao_portfolio(pd.read_csv('acoes.csv'), 5000, taxa_selic_historico.mean() / 100, 1000)

In [40]:
sharpe_ratio, melhores_pesos

(0.0961564547631217,
 array([0.00950142, 0.01290697, 0.00101367, 0.43584611, 0.1725474 ,
        0.36818443]))

In [41]:
_, _, acoes_pesos, soma_valor = alocacao_ativos(pd.read_csv('acoes.csv'), 5000, melhores_pesos=melhores_pesos)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



In [42]:
acoes_pesos, soma_valor

(   Ações      Pesos
 0   ABEV   0.950142
 1   ODPV   1.290697
 2    CCR   0.101367
 3  PETBR  43.584611
 4     BB  17.254740
 5   BOVA  36.818443, 13032.994532864475)

In [43]:
print(ls_retorno)

[0.04493260159751943, 0.06086318161755347, 0.02466950409303197, 0.06102914857072128, 0.023290586489327796, 0.06841140957330458, 0.06552618258554939, 0.04697337113831957, 0.02987845481204246, 0.033343641342997075, 0.09152945146250385, 0.04061157934872852, 0.048755635671080925, 0.07816639007068923, 0.09394868154849007, 0.06148842062029575, 0.06203344834147196, 0.05372564137306013, 0.06381252424031411, 0.06999179047706404, 0.04543703499655174, 0.07490151267327687, 0.06252918263891713, 0.03880069444644166, 0.08787233663110124, 0.0353476638877947, 0.05678406552593686, 0.061974005776077735, 0.05437144311546832, 0.029197666460661145, 0.06238361600324405, 0.0725142821502775, 0.08443881070238579, 0.07364386437850944, 0.027402963942517348, 0.08785216972449394, 0.06728828208369307, 0.06798760235930258, 0.0769913281343965, 0.059204597467397334, 0.04982276894658493, 0.031428252460396544, 0.03673516666406607, 0.09199666474085494, 0.008535570972534377, 0.022422627893837192, 0.023347814069752615, 0.09

In [44]:
print(ls_volatilidade)

[0.25976136311905745, 0.3182500648348988, 0.2416053865680047, 0.28641962339587096, 0.260079377732236, 0.2829037208900467, 0.27769132860507, 0.30790336014396097, 0.2831884050146637, 0.25747194505329063, 0.32214609113112486, 0.26918853804073883, 0.2756237108051183, 0.30125027955631084, 0.31365419919711446, 0.26385358924413826, 0.32700582205788614, 0.29139589779563346, 0.3153173079596498, 0.28834474272392596, 0.2836645484819027, 0.3157215093060713, 0.28918721613203296, 0.27197363482113984, 0.2988915916280809, 0.26328975347106626, 0.31538906771020675, 0.2791924949258211, 0.2866660929076804, 0.2544805360479827, 0.24972053190773963, 0.30328926921014554, 0.30995137894193847, 0.3039718508083982, 0.2871521052426242, 0.28146373309058936, 0.28027654949519276, 0.29850279302315186, 0.28303688644095776, 0.27705871755586414, 0.3002994742641825, 0.2547681067318351, 0.23765568750375382, 0.3095158938074298, 0.2495634682910302, 0.26238442972409826, 0.26358556388485577, 0.31382731051378837, 0.255211142734

In [45]:
print(ls_sharpe_ratio)

[-0.1654752059852957, -0.0850070056172587, -0.2617787768396153, -0.09387456689300636, -0.24848598432081173, -0.06894662619493432, -0.08063083638078164, -0.13297450053550552, -0.20494557978678166, -0.21195717192556357, 0.01121474044012741, -0.17573217515962367, -0.14208150264428743, -0.03236603335385408, 0.01923141758428242, -0.10016254136273053, -0.07915216359852115, -0.11733530071032756, -0.07644408289010599, -0.06216474079003649, -0.14975305126232594, -0.04122352646164646, -0.08778909513122625, -0.18059093210459728, -0.00014831476296791216, -0.1996621671972849, -0.09871173204182142, -0.09292033762398108, -0.11701845590089174, -0.23074063391211166, -0.10224650119220437, -0.05078446908623407, -0.011220650077934815, -0.04695435531349171, -0.21073745105584132, -0.00022914832211066308, -0.0736001089642621, -0.06676340983455514, -0.03860040530282333, -0.1036317119077115, -0.12685302834253173, -0.22172482627791765, -0.21535987857135613, 0.013181869350873197, -0.3180797904345577, -0.24961099

In [46]:
melhor_retorno, melhor_volatilidade

(0.12274967842451386, 0.3622534945122217)

Otimização de portfólio com algoritmos de otimização

In [47]:
!pip install mlrose

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=313763b7e6b86fbdc69c4cf9785309bf7bb9bc08fe09704b521831f38b857825
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [48]:
!pip install scikit-learn==0.22

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.0 MB 9.1 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires scikit-learn>=1.0.0, but you have scikit-learn 0.22 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.22 which is incompatible.


In [49]:
import mlrose

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



In [50]:
taxa_selic_historico

array([12.75, 14.25, 12.25,  6.5 ,  5.  ,  2.  ])

In [51]:
dataset_original = pd.read_csv('acoes.csv')
dinheiro_total = 5000
sem_risco = taxa_selic_historico.mean() / 100

In [52]:
def fitness_function(solucao):
  dataset = dataset_original.copy()
  pesos = solucao / solucao.sum()

  for i in dataset.columns[1:]:
    dataset[i] = (dataset[i] / dataset[i][0])

  for i, acao in enumerate(dataset.columns[1:]):
    dataset[acao] = dataset[acao] * pesos[i] * dinheiro_total

  dataset.drop(labels = ['Date'], axis = 1, inplace=True)
  dataset['soma valor'] = dataset.sum(axis = 1)
  dataset['taxa retorno'] = 0.0

  for i in range(1, len(dataset)):
    dataset['taxa retorno'][i] = ((dataset['soma valor'][i] / dataset['soma valor'][i - 1]) - 1) * 100

  sharpe_ratio = (dataset['taxa retorno'].mean() - sem_risco) / dataset['taxa retorno'].std() * np.sqrt(246)

  return sharpe_ratio

In [53]:
np.random.seed(10)
pesos = np.random.random(len(dataset_original.columns) - 1)
pesos = pesos / pesos.sum()
pesos

array([0.26617196, 0.00716121, 0.21866313, 0.25840174, 0.17202779,
       0.07757418])

In [54]:
fitness_function(pesos)

-0.2749940506901183

In [55]:
def visualiza_alocacao(solucao):
  colunas = dataset_original.columns[1:]
  for i in range(len(solucao)):
    print(colunas[i], solucao[i] * 100)

In [56]:
visualiza_alocacao(pesos)

ABEV 26.617195545614496
ODPV 0.7161207195395352
CCR 21.866313475981215
PETBR 25.84017365648649
BB 17.202779081759033
BOVA 7.757417520619242


In [57]:
fitness = mlrose.CustomFitness(fitness_function)

In [58]:
problema_maximizacao = mlrose.ContinuousOpt(length=6, fitness_fn=fitness,
                                            maximize = True, min_val = 0, max_val = 1)

In [59]:
problema_minimizacao = mlrose.ContinuousOpt(length=6, fitness_fn=fitness,
                                            maximize = False, min_val = 0, max_val = 1)

Algoritmo *Hill climb*

In [60]:
melhor_solucao, melhor_custo = mlrose.hill_climb(problema_maximizacao, random_state = 1)
melhor_solucao, melhor_custo

(array([0., 0., 0., 1., 0., 0.]), 0.13376976131421442)

In [61]:
melhor_solucao = melhor_solucao / melhor_solucao.sum()
melhor_solucao, melhor_solucao.sum()

(array([0., 0., 0., 1., 0., 0.]), 1.0)

In [62]:
visualiza_alocacao(melhor_solucao)

ABEV 0.0
ODPV 0.0
CCR 0.0
PETBR 100.0
BB 0.0
BOVA 0.0


In [63]:
_, _, _, soma_valor = alocacao_ativos(pd.read_csv('acoes.csv'), 5000, melhores_pesos=melhor_solucao)
soma_valor

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



16570.513672847515

In [64]:
pior_solucao, pior_custo = mlrose.hill_climb(problema_minimizacao, random_state = 1)
pior_solucao, pior_custo



(array([0.917022  , 0.42032449, 0.20011437, 0.        , 0.        ,
        0.        ]), -0.941150523569559)

In [65]:
pior_solucao = pior_solucao / pior_solucao.sum()
pior_solucao, pior_solucao.sum()

(array([0.59645226, 0.27338874, 0.130159  , 0.        , 0.        ,
        0.        ]), 1.0)

In [66]:
visualiza_alocacao(pior_solucao)

ABEV 59.645225503256974
ODPV 27.33887416805344
CCR 13.015900328689595
PETBR 0.0
BB 0.0
BOVA 0.0


In [67]:
_, _, _, soma_valor = alocacao_ativos(pd.read_csv('acoes.csv'), 5000, melhores_pesos=pior_solucao)
soma_valor

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



3503.3978535365454

Algoritmo *Simulated annealing*

In [68]:
melhor_solucao, melhor_custo = mlrose.simulated_annealing(problema_maximizacao, random_state = 1)
melhor_solucao = melhor_solucao / melhor_solucao.sum()
melhor_solucao, melhor_custo

(array([0., 0., 0., 1., 0., 0.]), 0.13376976131421442)

In [69]:
visualiza_alocacao(melhor_solucao)

ABEV 0.0
ODPV 0.0
CCR 0.0
PETBR 100.0
BB 0.0
BOVA 0.0


In [70]:
_, _, _, soma_valor = alocacao_ativos(pd.read_csv('acoes.csv'), 5000, melhores_pesos=melhor_solucao)
soma_valor

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



16570.513672847515

Algoritmo genético

In [71]:
problema_maximizacao_ag = mlrose.ContinuousOpt(length = 6, fitness_fn = fitness, 
                                               maximize = True, min_val = 0.1, max_val = 1)

In [72]:
melhor_solucao, melhor_custo = mlrose.genetic_alg(problema_maximizacao_ag, random_state = 1)
melhor_solucao = melhor_solucao / melhor_solucao.sum()
melhor_solucao, melhor_custo

(array([0.06543439, 0.04399487, 0.15797739, 0.31190652, 0.2946969 ,
        0.12598993]), -0.17306455963462553)

In [73]:
_, _, _, soma_valor = alocacao_ativos(pd.read_csv('acoes.csv'), 5000, melhores_pesos=melhor_solucao)
soma_valor

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



10307.836858756848